In [65]:
import pandas as pd

import InfoAPI
import requests
import json
import re

keys = InfoAPI.InfoAPI()

In [75]:
class Here:
    """Interacts with the HereAPI specifically for evacuation-routes"""
    address_1 = None
    address_2 = None
    lat_1 = None
    long_1 = None
    lat_2 = None
    long_2 = None
    geo = None
    summary = None
    set_map = None
    instructions = None
    testing = None
    
    def __init__(self, keys):
        self.keys = keys
        
    def main(self, a, b='Austin, TX', alt=0, input_self=False):
        """Gets travel summary"""
        if input_self:
            search = input("Address 1\n>>>> ")
            search_2 = input("Address 2\n>>>> ")
        else:
            search = a
            search_2 = b

        lat, long, lat_2, long_2 = self.get_geo(search, search_2)
        
        self.lat_1, self.long_1, self.lat_2, self.long_2 = lat, long, lat_2, long_2
        self.address_1 = a
        self.address_2 = b
        
        return self.get_route(lat, long, lat_2, long_2, alt=alt)
    
    def get_geo(self, search_text_1, search_text_2='Austin, TX'):
        """Calculates route

        Parameters
        ----------
        search_text_1 : string
            Address in format [number street, city, state zipcode]

        search_text_2 : string
            Address in format [number street, city, state zipcode]

        Returns
        -------
        lat : float
            First geographic latitude location.

        long : float
            First geographic longitude location.

        lat_2 : float
            Second geographic latitude location.

        long_2 : float
            Second geographic longitude location.

        Note
        ----
        If you only need one set of cordinates then do: get_get()[:2]

        """

        BASE_GEO = "https://geocoder.api.here.com/6.2/geocode.json"

        params_geo = {
            "app_id": self.keys.APP_ID,
            "app_code": self.keys.APP_CODE,
            "searchtext": search_text_1,
        }

        req = requests.get(BASE_GEO, params=params_geo)
        info = req.json()["Response"]

        lat, long = info['View'][0]["Result"][0]["Location"]["DisplayPosition"].values()

        params_geo['searchtext'] = search_text_2

        req = requests.get(BASE_GEO, params=params_geo)
        info = req.json()["Response"]

        lat_2, long_2 = info['View'][0]["Result"][0]["Location"]["DisplayPosition"].values()

        return (lat, long, lat_2, long_2)
    
    def get_route(self, lat, long, lat_2, long_2, alt=0):
        """Calculates route

        Parameters
        ----------
        lat : float
            First geographic latitude location.

        long : float
            First geographic longitude location.

        lat_2 : float
            Second geographic latitude location.

        long_2 : float
            Second geographic longitude location.

        Returns
        -------
        info : dict

        """
        BASE_ROUTING = "https://route.api.here.com/routing/7.2/calculateroute.json"

        params_routing = {
            "app_id": self.keys.APP_ID,
            "app_code": self.keys.APP_CODE,
            "waypoint0": "geo!" + str(lat) + "," + str(long),
            "waypoint1": "geo!" + str(lat_2) + "," + str(long_2),
            "mode": "fastest;car;traffic:enabled",
            "alternatives": alt
        }

        req = requests.get(BASE_ROUTING, params=params_routing)

        info = req.json()['response']

        return info
    
    def get_alt_routes(self, results, save_df=True, df_as='lat_long_routes.csv'):
        """Grabs all the geo locations for all the routes

        Parameters
        ----------
            results : dictionary
                From request.
            save_df : boolean, optional, default True
            df_as : string, optional, default 'routes.csv'

        Returns
        -------
            df : Pandas DataFrame
                Contains latitude and longitude from routes.

        """
        regex = r"(.*?)\<.*?\>"
        
        lat_long = []
        instructions = []
        self.testing = results['route']

        for count, j in enumerate(results['route']):
            for i in j['leg'][0]['maneuver']:
                instruction = i['instruction']
                
                lat_long.append([i['position']['latitude'], i['position']['longitude'], count])
                
                mystring = instruction
                result = re.findall(regex, mystring)
                instruction = ''.join(result)
                
                instructions.append([instruction, count])

        df = pd.DataFrame(lat_long, columns=['lat', 'long', 'route'])
        df_instruction = pd.DataFrame(instructions, columns=['instruction', 'route'])

        if save_df:
            df.to_csv(df_as, index=False)
            df_instruction.to_csv('instructions.csv', index=False)

        return df, df_instruction
    
    def get_summary(self, results, save_df=True, df_as='summary_routes.csv'):
        """Grabs the summary from each route

        Parameters
        ----------
            results : dictionary
                From request
            save_df : boolean, optional, default True
            df_as : string, optional, default 'routes.csv'

        Returns
        -------
            Pandas DataFrame

        """
        temp = []
        for i, route in enumerate(results['route']):
            traffic_time = (route['summary']['trafficTime'] - route['summary']['baseTime']) / 60
            total_time = route['summary']['travelTime'] / 60
            flags = ', '.join(route['summary']['flags'])

            temp.append([i, traffic_time, total_time, flags])

        df =  pd.DataFrame(temp, columns=['route', 'traffic_time', 'total_time', 'flags']).sort_values(by='total_time')

        if save_df:
            df.to_csv(df_as, index=False)

        return df
    
    def get_map(self):
        """UNAUTHORIZED"""
        # https://image.maps.api.here.com/mia/1.6/routing?c=1652B4&lw=6&t=0&ppi=320&w=400&h=600
        # BASE = 'https://image.maps.api.here.com/mia/1.6/routing'
        
        # params = {
            # "app_id": self.keys.APP_ID,
            # "app_code": self.keys.APP_CODE,
            # "waypoint0": [self.lat_1, self.long_1],
            # "waypoint1": [self.lat_2, self.long_2],
            # "c": "1652B4",
            # "lw": 6,
            # "t": 0,
            # "ppi": 320,
            # "w": 400,
            # "h": 600,
        # }
        
        # response = requests.get(BASE, params)
        # return response
    
    def fit(self, address_1, address_2, alt=0):
        """Grabs both dataframes

        Parameters
        ----------
            address_1 : string
            address_2 : string
            alt : int, optional, default = 0
                Alternate routes

        Returns
        -------
            geo : Pandas DataFrame
            summary : Pandas DataFrame

        """
        results = self.main(address_1, address_2, alt=alt)
        geo, instructions = self.get_alt_routes(results)
        summary = self.get_summary(results)
        
        self.geo = geo
        self.summary = summary
        self.instructions = instructions
        # self.set_map = self.get_map()


In [76]:
test = Here(keys)
test.fit('Killeen, TX', 'Austin, TX', alt=3)